In [4]:
import getpass
import openai

API_KEY = getpass.getpass("Enter your API key: ")



openai.api_key = API_KEY

In [5]:
from openai import OpenAI

client = OpenAI(
    api_key=API_KEY
)

In [6]:
from datasets import load_dataset

dataset = load_dataset("piuba-bigdata/contextualized_hate_speech")

In [7]:
dataset["train"][0]

{'id': 343726,
 'title': 'Video: salió de la cárcel por el coronavirus y murió de un tiro el mismo día al festejar su libertad',
 'text': '@usuario Uno menos',
 'context_tweet': 'Video: salió de la cárcel por el coronavirus y murió de un tiro el mismo día al festejar su libertad https://t.co/UrJGRAdEvD',
 'HATEFUL': 1,
 'body': 'Un hombre de 46 años, que cumplía una condena en prisión y fue puesto en libertad por el coronavirus, murió el mismo día que dejó la cárcel baleado por un primo, durante los "festejos" por dejar la prisión.\n\nSari Salem Wardat, de 46 años, recibió un disparo desde atrás en un aparente accidente mientras caminaba hacia miembros de la familia y vecinos que se habían reunido para darle la bienvenida a su casa, en Jordania.\n\nUn video que circuló en las redes sociales \u200bmuestra a Sari llegando a su casa en un auto azul rodeado de tiros de "celebración". El primo se acerca de inmediato a Sari, que sostiene una pistola y la apunta hacia el cielo.\n\nEl detenido

In [24]:
df_test = dataset["test"].to_pandas()

In [29]:

labels = df_test.columns[7:]

df_test["num_hatred"] = df_test[labels].sum(axis=1)

In [43]:

df_train = dataset["train"].to_pandas()

df_train["num_hatred"] = df_train[labels].sum(axis=1)


In [50]:
import pandas as pd

pd.options.display.max_colwidth = 1000
df_train[df_train["num_hatred"] > 1].sample(2)

def get_text_label(row):
    ret = ""
    for label in labels:
        if row[label] == 1:
            if ret == "":
                ret = label
            else:
                ret = ret + ", " + label
    if ret == "":
        ret = "nada"
    return ret

df_train["text_label"] = df_train.apply(get_text_label, axis=1)
df_test["text_label"] = df_test.apply(get_text_label, axis=1)

In [59]:
df_train[df_train["num_hatred"] >= 1].sample(2)

,id,title,text,context_tweet,HATEFUL,body,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL,num_hatred,text_label
8329,345474,Coronavirus en Argentina: se levantó la protesta de los presos en la cárcel de Melchor Romero,@usuario Ponga una bomba que mueran todo hay ratas,"Coronavirus en Argentina: 150 presos protestan en los techos de una cárcel y piden ""libertad"" para ""no morir"" en prisión https://t.co/RcvBqs0bS3",1,"Las autoridades del Servicio Penitenciario Bonaerense (SPB) informaron esta tarde que se ""retomó la calma"" en la Unidad 10 de Melchor Romero, donde este martes, bien temprano, un grupo de presos inició una protesta para reclamar salidas transitorias y otros beneficios en medio de la pandemia por el coronavirus .\n\nHoras antes, entre 10 y 20 detenidos se subieron a los techos del edificio y, a través de videos enviados con celulares, plantearon pedidos a las autoridades penales y judiciales.\n\nA partir de la intervención de representantes de la Defensoría General de La Plata, la Comisión Provincial por la Memoria (CPM) y directivos del SPB lograron desactivar la manifestación y recuperar el control del edificio que está ubicado en 519 y 176 de esa localidad perteneciente al partido de La Plata.​\n\n""La situación quedó controlada sin que se registraran heridos o lesionados. Los internos preparan un petitorio para el Poder Judicial"", dijeron esta tarde fuentes penitenciarias.\n\nPro...",1,0,0,0,0,0,0,0,1,1,CRIMINAL
3353,337615,"Wuhan celebra el fin de la cuarentena por el coronavirus con un mensaje para el resto del mundo: ""Aprendan de nuestros errores""",@usuario Chinos culiados de mierda ojalá se mueran todos,"Wuhan celebra el fin de la cuarentena por el coronavirus con un mensaje para el resto del mundo: ""Aprendan de nuestros errores"" https://t.co/mBLG1XfOGD",1,"Miles de chinos celebraron con euforia este miércoles 8 de abril el final de más de dos meses de confinamiento en Wuhan, cuna de la pandemia de coronavirus. Mientras tanto, el virus no da tregua a Estados Unidos y Europa, donde no se logra generar una respuesta económica unificada y las muertes volvieron a aumentar en España​.\n\nEn medio de la aterradora marcha del nuevo coronavirus, el mundo vio una luz de esperanza con la imágenes de miles de pasajeros que a partir de la medianoche del martes empezaron a tomar por asalto estaciones de micros y trenes, algunos de ellos con atuendos de protección integral, en Wuhan, la capital de la provincia china de Hubei.\n\n""Me levanté a las cuatro hoy. ¡Me siento tan bien!"", dijo una feliz Hao Mei, de 39 años y oriunda de Enshi, ciudad a 450 kilómetros al oeste de Wuhan. Estaba a punto de subirse a un tren para regresar con sus dos hijos, a los que tuvo que dejar solos durante más de dos meses, ya que quedó atrapada a fines de enero en Wuhan....",1,0,0,1,0,0,0,0,0,1,RACISM


In [75]:
template_prompt = f"""Determinar si el siguiente mensaje contiene discurso discriminatorio o no. Entendemos que hay discurso discriminatorio en el tweet si contiene declaraciones de carácter intenso y posiblemente irracional de rechazo, enemistad y aborrecimiento contra un individuo o contra un grupo, siendo estos objetivos de estas expresiones por poseer una característica protegida.

Las características protegidas que contemplamos son:

- Mujeres (mujer)
- Género o identidad sexual (lgbti)
- Inmigrantes, extranjeros, religión, pueblos aborígenes u otras nacionalidades (racismo)
- Clase, situación socioeconómica o barrio de residencia (clase)
- Poseer discapacidades, problemas salud mental o de adicción al alcohol, drogas u otros estupefacientes (discapacidad)
- Opinión o ideología política (política)
- Aspecto o edad (aspecto)
- Antecedentes penales o estar privado de la libertad (criminal)

En contexto daremos un titular de una noticia, y en texto un comentario a analizar. Responder una, varias o ninguna de las características anteriores.
###
contexto: Al borde del llanto, Nati Jota pidió disculpas por sus repudiables tuits pero los justificó: 'Quería sumar seguidores'
texto: Show o nada mi amorrr!!
La imbesilidad al palo!
salida: nada
###
contexto: Violento motín en la cárcel de Devoto: tomaron e incendiaron pabellones
texto: Hermanito lo que son esos grones. Ahí adentro le robaron la billetera al coronavirus
salida: criminal, racismo, aspecto
###
contexto: Mayra Mendoza criticó el banderazo contra el Gobierno: "No tiene sentido y es antidemocrático"
texto: Que negrita villera....ni con la plata q roba puede arreglarse
salida: mujer, racismo, aspecto, clase
###
contexto: No da tregua: sin ropa y desafiante Flor Peña volvió a mostrar su lado más hot
texto: K lado hot tiene más varices k mi abuela. Tiene mas puesta de espalda k karagian trola
salida: mujer, aspecto
###
contexto: Wuhan celebra el fin de la cuarentena por el coronavirus con un mensaje para el resto del mundo: "Aprendan de nuestros errores"
texto: Chinos culiados de mierda ojalá se mueran todos
salida: racismo
###
"""



In [76]:
import re
from pysentimiento.preprocessing import preprocess_tweet

url_regex = r'\burl\b'



def my_preprocess(text):
    text = preprocess_tweet(text, preprocess_hashtags=False, demoji=False)
    text = text.replace("@usuario", "")

    text = re.sub(url_regex, "", text)
    # Replace multiple spaces with one
    text = " ".join(text.split())
    text = text.replace("\n", " ")

    return text

my_preprocess("@usuario Vergonzoso es lo que dice este sr. Los planeros paraguayos, bolivianos, peruanos como cobran el IFE, cobran jubilaciones, votan en Argentina ??? No me diga que NO sabe... es una burla! 👎🤦💩🤡 http://google.com")

'Vergonzoso es lo que dice este sr. Los planeros paraguayos, bolivianos, peruanos como cobran el IFE, cobran jubilaciones, votan en Argentina ??? No me diga que NO sabe... es una burla! 👎🤦💩🤡'

In [80]:
df_test.shape

(11343, 17)

In [84]:
sample_df = df_test.sample(100)

In [ ]:
outs = {

}

In [98]:
from tqdm import tqdm
import time


def get_completion(prompt):
    return client.chat.completions.create(messages=[{
            "role": "user",
            "content": prompt
    }], model="gpt-3.5-turbo")

for idx, row in tqdm(sample_df.iterrows(), total=sample_df.shape[0]):

    if idx in outs:
        continue

    contexto = preprocess_tweet(row["context_tweet"], preprocess_hashtags=False, demoji=False, preprocess_handles=False)
    texto = my_preprocess(row["text"])
    prompt = template_prompt + f"contexto: {contexto}\ntexto: {texto}\nsalida: "

    response = get_completion(prompt)

    outs[idx] = response.choices[0].message.content

    time.sleep(0.1)

100%|██████████| 100/100 [00:36<00:00,  2.72it/s]


In [102]:
import json

with open("outputs.json", "w") as f:
    json.dump(outs, f)


In [108]:
labels = []

Index(['WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED',
       'APPEARANCE', 'CRIMINAL'],
      dtype='object')

In [106]:
from collections import Counter

counter = Counter()


for v in outs.values():
    for cl in v.split(","):
        counter[cl.strip().lower()] += 1


In [107]:

counter

Counter({'política': 46,
         'nada': 24,
         'ninguna': 14,
         'mujer': 13,
         'clase': 7,
         'racismo': 6,
         'criminal': 4,
         'aspecto': 4,
         'discapacidad': 2,
         'inmigrantes': 1,
         'género o identidad sexual': 1,
         'opinión-política': 1,
         'lgbti': 1})

In [112]:
sample_df["pred"] = sample_df.index.map(outs)


In [115]:
sample_df.to_csv("sample_df.csv", index=False)